<div class="container" style="display:flex; width:100%;height:150px;background-color:None;">
    <div class="subcontainer-1" style="flex:0.11;height:100%;width:20%;">
        <div class="logo-frame" style="height:100%;background-color:None;">
            <img src="https://github.com/andikaaa18/SkillAcademy_Project/blob/main/skill_academy_logo.jpg?raw=true" style="height:100%;float:left; filter:invert(0%);">
        </div>
    </div>
    <div class="subcontainer-2" style="flex:0.89;height:60%;margin-top:2.5%">
        <div class="upper-sidebar" style="height:50%;width:100%;background-color:None;align-items:center;display:inline-flex;">
            <div style="font-size:50px; font-family:Trebuchet MS; font-weight:bold; text-align:left">
            Skill Academy Project
            </div>
        </div>
        <div class="lower-sidebar" style="height:50%;width:100%;background-color:None;align-items:center;display: inline-flex;">
            <div style="font-size:30px; font-family:Trebuchet MS;">
            &mdash; Final Project : Stock Price Forecasting
            </div>
        </div>
    </div>
</div>

<hr style="width:95%;height:2px;background-color:black;" align="left">

<div style="font-family:Trebuchet MS; font-weight:bold; font-size:45px; text-align:left; line-height:50px">
A Comprehensive Analysis for UNTR Stock Predictions with LSTM Neural Networks
</div>

&mdash; Deployment model

In [278]:
# library yang digunakan
import torch
import time
import datetime
import pandas as pd
import joblib
import gradio as gr

In [279]:
# function untuk query historical data harga saham UNTR

def query_historical_data(n_days):
    """
    Args:
        n_days (int): Jumlah hari yang akan diambil datanya terhitung dari hari kemarin.
    """
    current_time = datetime.datetime.now()
    
    ticker = 'UNTR.JK'
    period1 = int(time.mktime((datetime.datetime.now() - datetime.timedelta(days=n_days)).timetuple()))
    period2 = int(time.mktime((datetime.datetime.now() - datetime.timedelta(days=1)).timetuple()))
    interval = '1d' # '1wk', '1m', dll.

    query_str = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true"

    return pd.read_csv(query_str, parse_dates=['Date'], index_col='Date')


In [280]:
# membuat function untuk data preprocessing
def data_preprocessing(data, n=7):
    
    def make_features(data:pd.DataFrame, lag:list[int]=None):
        df = data.copy()
        
        # lag features
        if lag: # will be executed if the variable has a value
            for i in lag:
                df[f"Open(t-{i})"] = df['Open'].shift(i)
                df[f"High(t-{i})"] = df['High'].shift(i)
                df[f"Low(t-{i})"] = df['Low'].shift(i)
                df[f"Close(t-{i})"] = df['Close'].shift(i)
        
        return df.dropna()

    def remove_unimportant_features(data:pd.DataFrame):
        
        return data.drop(columns=["Volume", "Adj Close", 'Open', 'High', 'Low', 'Close'])

    def features_scaling(data):
        scaler = joblib.load("assets/scaler.pkl")
        return scaler.transform(data)
    
    def reshape_data(data):
        return torch.tensor(data.reshape((-1, data.shape[1], 1)), dtype=torch.float)
        
    data_prep = make_features(data, lag=list(range(1,n+1)))
    data_prep = remove_unimportant_features(data_prep)[-1:]
    data_prep = features_scaling(data_prep)
    data_prep = reshape_data(data_prep)
    
    return data_prep

In [281]:
def deploy(n_forecast):
    # memuat model
    path = "model/untr_stock_price_predictor.pt"
    model = torch.jit.load(path)
    
    # query historical data      
    data = query_historical_data(50)

    # data pre-processing
    data_preprocessing(data, n=7).shape
    
    # forecasting
    result = []
    # output_box = gr.Group()
    with torch.inference_mode():
        for i in range(n_forecast):
            predict = model(data_preprocessing(data, n=7))
            result.append({
                "Open": predict[:,0].item(),
                "High": predict[:,1].item(),
                "Low": predict[:,2].item(),
                "Close": predict[:,3].item()
            })
            
            # modif data: memasukkan hasil forecasting terakhir
            pred_df = pd.DataFrame(data=torch.concat([predict, torch.tensor([[0,0]])], axis=1), columns=data.columns)
            data = pd.concat([data, pred_df, pred_df]) # --> alasan concat pred_df 2 kali agar data nya tidak hilang saat membuat lag-features
    
    return result
    # return output_box

In [284]:
# membuat interface
interface = gr.Interface(fn=deploy,
                         inputs=gr.Slider(minimum=1,maximum=5,step=1, label="n-forecasting"),
                         outputs=gr.Json())

interface.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "C:\Users\andik\AppData\Local\Programs\Python\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\andik\AppData\Local\Programs\Python\Python310\lib\asyncio\proactor_events.py", line 162, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
